In [101]:
import numpy as np
import scipy.linalg as sal

In [102]:
## The eigen function in product bases is known, now we have apply S^2 operator 
# on the two particle system eigen functions, then form the matrix 
# and find the coupled bases as the eigen vector of the corresponding matrix of S^2.

In [103]:
h_cut = 1
m_e = 1

In [104]:
def inner_product(left_vec, right_vec):
    bra = (np.conj(left_vec)).T
    ket = right_vec
    inner_prod = bra @ ket
    return inner_prod

In [105]:
def norm_column_vec(x):
    # m, n = x.shape
    inner_pro = inner_product(x, x)
    # inner_prodcut is still a 1x1 array, lets extract the element and take its sqrt
    norm = np.sqrt(np.squeeze(inner_pro))
    x_norm = x / norm
    return x_norm

In [106]:
def product_bases(eig_vec):
    up_up = eig_vec[0].reshape(-1,1)
    up_down = eig_vec[1].reshape(-1,1)
    down_up = eig_vec[2].reshape(-1,1)
    down_down = eig_vec[3].reshape(-1,1)
    
    function_list = [up_up, up_down, down_up, down_down]
    function_dict = {"up_up": up_up,
                "up_down": up_down,
                "down_up": down_up,
                "down_down": down_down}
    return function_list, function_dict

In [107]:
def individual_spin_operation_matrix():
    S_1_2 = np.array([[3/4,0,0,0],[0,3/4,0,0],[0,0,3/4,0], [0,0,0,3/4]])
    S_1z = np.array([[1/2,0,0,0],[0,1/2,0,0],[0,0,-1/2,0], [0,0,0,-1/2]])
    S_2z = np.array([[1/2,0,0,0],[0,-1/2,0,0],[0,0,1/2,0], [0,0,0,-1/2]])
    S_1_plus = np.array([[0,0,1,0],[0,0,0,1],[0,0,0,0], [0,0,0,0]])
    S_2_plus = np.array([[0,1,0,0],[0,0,0,0],[0,0,0,1], [0,0,0,0]])
    S_1_minus = np.array([[0,0,0,0],[0,0,0,0],[1,0,0,0], [0,1,0,0]])
    S_2_minus = np.array([[0,0,0,0],[1,0,0,0],[0,0,0,0], [0,0,1,0]])
    
    operate = {"S_1_2": S_1_2,
               "S_2_2": S_1_2,
               "S_1z": S_1z,
               "S_2z" : S_2z,
               "S_1_plus": S_1_plus,
               "S_2_plus" : S_2_plus,
               "S_1_minus": S_1_minus,
               "S_2_minus" : S_2_minus
              }
    return operate

In [108]:
# Solve this to find eigen_vectors corresponding to the product bases; S_1^2 = S_2^2
# representation S_1^2 = S_1_2; and S_2^2 = S_2_2; since S_1_2 = S_2_2
operate = individual_spin_operation_matrix()
S_1_2 = operate["S_1_2"]
val, vec = sal.eig(S_z)
function_list, function_dict = product_bases(vec)

In [111]:
def S_2_operate(function_list):
    l = len(function_list)
    mat = np.zeros((l,l))
    operate = individual_spin_operation_matrix()
    S_1_2 = operate["S_1_2"]
    S_2_2 = operate["S_2_2"]
    S_1_plus = operate["S_1_plus"]
    S_2_plus = operate["S_2_plus"]
    S_1_minus = operate["S_1_minus"]
    S_2_minus = operate["S_2_minus"]
    S_1z = operate["S_1z"]
    S_2z = operate["S_2z"]
    
    S_2_operation = S_1_2 + S_2_2 + 2*(0.5*(S_1_plus@S_2_minus+ S_1_minus@S_2_plus) + S_1z@S_2z)
    
    for i in range(l):
        for j in range(l):
            mat[i,j] = np.squeeze(function_list[i].T @ S_2_operation @ function_list[j])
            
    return mat

In [113]:
S_2_mat = S_2_operate(function_list)
S_2_mat

array([[2., 0., 0., 0.],
       [0., 1., 1., 0.],
       [0., 1., 1., 0.],
       [0., 0., 0., 2.]])

In [114]:
# Solving for the coupled bases and clebsch gordon coeff
coupled_val ,coupled_vec = sal.eig(S_2_mat)

In [117]:
coupled_val

array([2.+0.j, 0.+0.j, 2.+0.j, 2.+0.j])

In [120]:
CG_matrix = coupled_vec.T
CG_matrix

array([[ 0.        ,  0.70710678,  0.70710678,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

From the coupled value, we can see that second eigen vector corresponds to a singlet state
wehre as all other vectors are for triplet state: